# Merging gold spans with debate metadata and turn context 📋

This notebook loads the consolidated gold-span table and merges it with full debate texts to extract speaker turns, previous/next turn snippets, and associated metadata (e.g., SessionID, Speaker, Utterance).  
It then attempts to match each span’s surrounding turn to its corresponding anonymized debate record, flags unmatched cases, and summarizes label distributions and match rates.

In [ ]:
import pandas as pd

# An early version of the data
df = pd.read_csv('/Users/pbrams/Desktop/AARHUS_UNIVERSITY/kandidat/thesis_work/annotations/output/coalesced_gold_aug_three_levels_18_april.csv')

df['debate_unit_id'] = df['debate_unit_id'].astype(int)

# Get context df for full_text
df_context = pd.read_csv('/Users/pbrams/Desktop/AARHUS_UNIVERSITY/kandidat/thesis_work/annotations/output/df_all.csv')

df_context_unique = df_context.drop_duplicates(subset="debate_unit_id", keep="first")

df = df.merge(
    df_context_unique[["debate_unit_id", "full_text"]],
    on="debate_unit_id",
    how="left"
)

df_context = pd.read_csv('/Users/pbrams/Desktop/AARHUS_UNIVERSITY/kandidat/thesis_work/annotations/output/df_all.csv')
df_context

In [ ]:
# Get previous and next turns on 
import pandas as pd
import re

# Function to extract structured turns from full_text
def extract_turns(full_text):
    pattern = r'\*\*(.*?)\*\*:'
    split = re.split(pattern, full_text)
    speaker_turns = []
    for i in range(1, len(split) - 1, 2):
        speaker = split[i].strip().lower()
        utterance = split[i + 1].strip()
        speaker_turns.append((speaker, utterance))
    return speaker_turns

# Updated function to include speakers for prev/next turns
def extract_span_context(row):
    full_text = row['full_text']
    span = row['span'].strip()
    turns = extract_turns(full_text)

    for idx, (speaker, utterance) in enumerate(turns):
        if span in utterance:
            return pd.Series({
                'span_turn': utterance,
                'span_speaker': speaker,
                'prev_turn': turns[idx - 1][1] if idx > 0 else '',
                'prev_turn_speaker': turns[idx - 1][0] if idx > 0 else '',
                'next_turn': turns[idx + 1][1] if idx < len(turns) - 1 else '',
                'next_turn_speaker': turns[idx + 1][0] if idx < len(turns) - 1 else ''
            })

    # If no match is found
    return pd.Series({
        'span_turn': '',
        'span_speaker': '',
        'prev_turn': '',
        'prev_turn_speaker': '',
        'next_turn': '',
        'next_turn_speaker': ''
    })

# Apply to dataframe
#context_features = df_features.apply(extract_span_context, axis=1)
context_features = df.apply(extract_span_context, axis=1)

# Combine with original dataframe
df = pd.concat([df, context_features], axis=1)

# pct no span_turn but there is contents in span col
mask_no_span_turn = (
    (df['span_turn'].fillna("") == "") &  # no content in span_turn
    (df['span'].fillna("") != "")         # but there IS content in span
)

pct_no_span_turn = mask_no_span_turn.mean() * 100
print(f"Percentage of rows with no content in span_turn but content in span: {pct_no_span_turn:.2f}%")

#df.to_csv("/Users/pbrams/Desktop/AARHUS_UNIVERSITY/kandidat/thesis_work/annotations/output/coalesced_gold_w_turns_df.csv")
df.to_csv("/Users/pbrams/Desktop/AARHUS_UNIVERSITY/kandidat/thesis_work/annotations/output/coalesced_gold_aug_w_turns_df.csv")

df

In [20]:
len(df['debate_unit_id'].unique())

199

## Getting metadata on

In [ ]:
import pandas as pd 

# Coalesced data
df = pd.read_csv("/Users/pbrams/Desktop/AARHUS_UNIVERSITY/kandidat/thesis_work/annotations/output/coalesced_gold_aug_w_turns_df.csv")
df = df.drop(columns=[col for col in df.columns if col.startswith("Unnamed:")])

df

In [22]:
import pandas as pd

# Get old df in with meta-data
#debates = pd.read_csv("/Users/pbrams/Desktop/AARHUS_UNIVERSITY/kandidat/thesis_work/annotations/src/data_from_other_nb/coalesced_gold_df_with_turns_and_anon_6_april.csv")
debates = pd.read_csv("/Users/pbrams/Desktop/AARHUS_UNIVERSITY/kandidat/thesis_work/annotations/src/data_from_other_nb/coalesced_gold_df_with_turns_and_anon_18_april.csv")

debates["debate_unit_id"] = debates["debate_unit_id"].astype(int)
debates = debates.drop(columns=[col for col in debates.columns if col.startswith("Unnamed:")])

debates


,SessionID,MeetingNumber,Date,Location,AgendaItemNo,AgendaTitle,DebateType,TurnNo,Speaker,Party,...,TurnRole,Time,Utterance,AgendaCategory,MeetingDateID,AgendaTitleDateID,TurnSequence,debate_unit_id,TurnRole_Danish,Utterance_anon
0,20091,4,2009-10-20 13:00:00,Folketingssalen,6,1. behandling af L 27: Om Europol.,reading of bill,6,Dennis Flydtkjær,DF,...,proponent,NaN,Dette lovforslag handler om at gennemføre Råde...,Foreign Affairs,4_2009-10-20 13:00:00,1. behandling af L 27: Om Europol._2009-10-20 ...,0,187,Ordfører,Dette lovforslag handler om at gennemføre Råde...
1,20091,4,2009-10-20 13:00:00,Folketingssalen,6,1. behandling af L 27: Om Europol.,reading of bill,8,Karen Hækkerup,S,...,asker,NaN,"[Lydudfald] … den her frygt, som Parti_F har f...",Foreign Affairs,4_2009-10-20 13:00:00,1. behandling af L 27: Om Europol._2009-10-20 ...,1,187,Spørger,"[Lydudfald] … den her frygt, som Parti_F har f..."
2,20091,4,2009-10-20 13:00:00,Folketingssalen,6,1. behandling af L 27: Om Europol.,reading of bill,10,Dennis Flydtkjær,DF,...,proponent,NaN,"Jeg er ikke overrasket over, at Parti_A er uen...",Foreign Affairs,4_2009-10-20 13:00:00,1. behandling af L 27: Om Europol._2009-10-20 ...,2,187,Ordfører,"Jeg er ikke overrasket over, at Parti_A er uen..."
3,20091,4,2009-10-20 13:00:00,Folketingssalen,6,1. behandling af L 27: Om Europol.,reading of bill,12,Karen Hækkerup,S,...,asker,NaN,Nu samarbejder man jo også med politimyndighed...,Foreign Affairs,4_2009-10-20 13:00:00,1. behandling af L 27: Om Europol._2009-10-20 ...,3,187,Spørger,Nu samarbejder man jo også med politimyndighed...
4,20091,4,2009-10-20 13:00:00,Folketingssalen,6,1. behandling af L 27: Om Europol.,reading of bill,14,Dennis Flydtkjær,DF,...,proponent,NaN,"Jeg kan meddele Spørgeren, at jeg bestemt er e...",Foreign Affairs,4_2009-10-20 13:00:00,1. behandling af L 27: Om Europol._2009-10-20 ...,4,187,Ordfører,"Jeg kan meddele Spørgeren, at jeg bestemt er e..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301200,20121,68,2013-03-13 13:00:00,Folketingssalen,2,Besvarelse af oversendte spørgsmål til ministrene,other,153,Manu Sareen,NaN,...,minister,NaN,"Nu er det god latin herinde, at man forholder ...",Other,68_2013-03-13 13:00:00,Besvarelse af oversendte spørgsmål til ministr...,48,27880,Minister,"Nu er det god latin herinde, at man forholder ..."
301201,20191,88,2020-03-31 16:00:00,Folketingssalen,0,Punkt 0,other,2,MødeSlut MødeSlut,MødeSlut,...,unknown,(Kl. 22:44),Mødet er hævet. .,Other,88_2020-03-31 16:00:00,Punkt 0_2020-03-31 16:00:00,0,81458,Ukendt,Mødet er hævet. .
301202,20191,88,2020-03-31 16:00:00,Folketingssalen,0,Punkt 0,other,4,Pause Pause,Pause,...,unknown,(Kl. 18:30),Mødet er udsat. .,Other,88_2020-03-31 16:00:00,Punkt 0_2020-03-31 16:00:00,1,81458,Ukendt,Mødet er udsat. .
301203,20191,88,2020-03-31 16:00:00,Folketingssalen,0,Punkt 0,other,6,Pause Pause,Pause,...,unknown,(Kl. 20:30),Mødet er udsat. .,Other,88_2020-03-31 16:00:00,Punkt 0_2020-03-31 16:00:00,2,81458,Ukendt,Mødet er udsat. .


In [23]:
debates.columns

Index(['SessionID', 'MeetingNumber', 'Date', 'Location', 'AgendaItemNo',
       'AgendaTitle', 'DebateType', 'TurnNo', 'Speaker', 'Party', 'Role',
       'TurnRole', 'Time', 'Utterance', 'AgendaCategory', 'MeetingDateID',
       'AgendaTitleDateID', 'TurnSequence', 'debate_unit_id',
       'TurnRole_Danish', 'Utterance_anon'],
      dtype='object')

Plan now is to for each row in coalesced_gold_w_turns_df (df_coal below):

1. Get the current debate_unit_id

2. Subset df_debates by that debate_unit_id

3. Look for the row in that subset whose Utterance_anon contains the string in span_turn

4. If a match is found, copy the desired metadata columns (prefixed with "METADATA_") into df_coal for that row

5. Otherwise, set a flag column (MATCH_FLAG) to "NO MATCH"

This approach will keep the row count in df_coal identical but add new columns for the matched metadata.

In [25]:
import pandas as pd
import numpy as np

# df = pd.read_csv("coalesced_gold_w_turns_df.csv") 
# debates = pd.read_csv("df_debates.csv")

###############################################################################
# Define metadata columns
###############################################################################
metadata_cols = [
    'SessionID', 'MeetingNumber', 'Date', 'Location', 'AgendaItemNo',
    'AgendaTitle', 'DebateType', 'TurnNo', 'Speaker', 'Party', 'Role',
    'TurnRole', 'Time', 'Utterance', 'AgendaCategory', 'MeetingDateID',
    'AgendaTitleDateID', 'TurnRole_Danish'
]

###############################################################################
# 1) Create the needed metadata columns for the "current turn" snippet
###############################################################################
for col in metadata_cols:
    df['METADATA_' + col] = ""
df['MATCH_FLAG'] = "NO MATCH"

###############################################################################
# PASS 1: Current turn snippet in 'span' -> check 'Utterance_anon'
###############################################################################
for i, row in df.iterrows():
    debate_id = row['debate_unit_id']
    snippet = row['span']  # current turn snippet

    if pd.isna(snippet):
        snippet_str = ""
    else:
        snippet_str = str(snippet).strip()

    if snippet_str == "":
        continue  # remains NO MATCH

    # Subset debates to matching debate_id
    subset_debate = debates[debates['debate_unit_id'] == debate_id]

    # Attempt match in 'Utterance_anon'
    match = subset_debate[
        subset_debate['Utterance_anon'].astype(str).str.contains(snippet_str, na=False, regex=False)
    ]

    if not match.empty:
        first_match = match.iloc[0]
        for col in metadata_cols:
            df.at[i, 'METADATA_' + col] = first_match[col]
        df.at[i, 'MATCH_FLAG'] = "MATCH"

###############################################################################
# PASS 2: Current turn snippet in 'span' -> check 'Utterance' if still NO MATCH
###############################################################################
no_match_indices = df.index[df['MATCH_FLAG'] == "NO MATCH"]

for i in no_match_indices:
    row = df.loc[i]
    debate_id = row['debate_unit_id']
    snippet = row['span']

    if pd.isna(snippet):
        snippet_str = ""
    else:
        snippet_str = str(snippet).strip()

    if snippet_str == "":
        continue

    subset_debate = debates[debates['debate_unit_id'] == debate_id]

    # Attempt match in 'Utterance'
    match = subset_debate[
        subset_debate['Utterance'].astype(str).str.contains(snippet_str, na=False, regex=False)
    ]

    if not match.empty:
        first_match = match.iloc[0]
        for col in metadata_cols:
            df.at[i, 'METADATA_' + col] = first_match[col]
        df.at[i, 'MATCH_FLAG'] = "MATCH"

###############################################################################
# 2) Create the needed metadata columns for the "previous turn"
###############################################################################
for col in metadata_cols:
    df['METADATA_PREV_' + col] = ""
df['PREV_MATCH_FLAG'] = "NO MATCH"

###############################################################################
# PASS 1: Previous turn snippet in 'prev_turn' -> check 'Utterance_anon'
###############################################################################
for i, row in df.iterrows():
    debate_id = row['debate_unit_id']
    snippet = row['prev_turn']

    if pd.isna(snippet):
        snippet_str = ""
    else:
        snippet_str = str(snippet).strip()

    if snippet_str == "":
        continue

    subset_debate = debates[debates['debate_unit_id'] == debate_id]

    match = subset_debate[
        subset_debate['Utterance_anon'].astype(str).str.contains(snippet_str, na=False, regex=False)
    ]

    if not match.empty:
        first_match = match.iloc[0]
        for col in metadata_cols:
            df.at[i, 'METADATA_PREV_' + col] = first_match[col]
        df.at[i, 'PREV_MATCH_FLAG'] = "MATCH"

###############################################################################
# PASS 2: Previous turn snippet in 'prev_turn' -> check 'Utterance'
###############################################################################
no_match_indices = df.index[df['PREV_MATCH_FLAG'] == "NO MATCH"]

for i in no_match_indices:
    row = df.loc[i]
    debate_id = row['debate_unit_id']
    snippet = row['prev_turn']

    if pd.isna(snippet):
        snippet_str = ""
    else:
        snippet_str = str(snippet).strip()

    if snippet_str == "":
        continue

    subset_debate = debates[debates['debate_unit_id'] == debate_id]

    match = subset_debate[
        subset_debate['Utterance'].astype(str).str.contains(snippet_str, na=False, regex=False)
    ]

    if not match.empty:
        first_match = match.iloc[0]
        for col in metadata_cols:
            df.at[i, 'METADATA_PREV_' + col] = first_match[col]
        df.at[i, 'PREV_MATCH_FLAG'] = "MATCH"

###############################################################################
# 3) Create the needed metadata columns for the "next turn"
###############################################################################
for col in metadata_cols:
    df['METADATA_NEXT_' + col] = ""
df['NEXT_MATCH_FLAG'] = "NO MATCH"

###############################################################################
# PASS 1: Next turn snippet in 'next_turn' -> check 'Utterance_anon'
###############################################################################
for i, row in df.iterrows():
    debate_id = row['debate_unit_id']
    snippet = row['next_turn']

    if pd.isna(snippet):
        snippet_str = ""
    else:
        snippet_str = str(snippet).strip()

    if snippet_str == "":
        continue

    subset_debate = debates[debates['debate_unit_id'] == debate_id]

    match = subset_debate[
        subset_debate['Utterance_anon'].astype(str).str.contains(snippet_str, na=False, regex=False)
    ]

    if not match.empty:
        first_match = match.iloc[0]
        for col in metadata_cols:
            df.at[i, 'METADATA_NEXT_' + col] = first_match[col]
        df.at[i, 'NEXT_MATCH_FLAG'] = "MATCH"

###############################################################################
# PASS 2: Next turn snippet in 'next_turn' -> check 'Utterance'
###############################################################################
no_match_indices = df.index[df['NEXT_MATCH_FLAG'] == "NO MATCH"]

for i in no_match_indices:
    row = df.loc[i]
    debate_id = row['debate_unit_id']
    snippet = row['next_turn']

    if pd.isna(snippet):
        snippet_str = ""
    else:
        snippet_str = str(snippet).strip()

    if snippet_str == "":
        continue

    subset_debate = debates[debates['debate_unit_id'] == debate_id]

    match = subset_debate[
        subset_debate['Utterance'].astype(str).str.contains(snippet_str, na=False, regex=False)
    ]

    if not match.empty:
        first_match = match.iloc[0]
        for col in metadata_cols:
            df.at[i, 'METADATA_NEXT_' + col] = first_match[col]
        df.at[i, 'NEXT_MATCH_FLAG'] = "MATCH"

###############################################################################
# Save final and show no-match percentages
###############################################################################
df.to_csv("output/coalesced_gold_aug_w_turns_w_context.csv", index=False)

pct_no_match_current = (df['MATCH_FLAG'].eq("NO MATCH").mean()) * 100
pct_no_match_prev    = (df['PREV_MATCH_FLAG'].eq("NO MATCH").mean()) * 100
pct_no_match_next    = (df['NEXT_MATCH_FLAG'].eq("NO MATCH").mean()) * 100

print(f"Percentage 'NO MATCH' for current turn  : {pct_no_match_current:.2f}%")
print(f"Percentage 'NO MATCH' for previous turn : {pct_no_match_prev:.2f}%")
print(f"Percentage 'NO MATCH' for next turn     : {pct_no_match_next:.2f}%")


Percentage 'NO MATCH' for current turn  : 0.00%
Percentage 'NO MATCH' for previous turn : 1.85%
Percentage 'NO MATCH' for next turn     : 41.75%


Percentage 'NO MATCH' for current turn  : 0.00%
Percentage 'NO MATCH' for previous turn : 3.26%
Percentage 'NO MATCH' for next turn     : 40.76%

Then 
Percentage 'NO MATCH' for current turn  : 0.00%
Percentage 'NO MATCH' for previous turn : 1.82%
Percentage 'NO MATCH' for next turn     : 40.09%

In [26]:
# check the label distribution

# Total unique labeled spans
num_unique = len(df)
print(f"🔢 Total unique labeled spans: {num_unique}")

# Label distribution (as % of unique spans)
label_dist = (
    df['label']
    .value_counts(normalize=True)
    .mul(100.0)
    .round(2)
    .reset_index()
    .rename(columns={'index': 'Label', 'label': 'Percentage'})
)
print("\n📊 Label Distribution (% of unique labeled spans):")
print(label_dist)

# Distribution of quality_status (percentage of each category)
quality_status_dist = (
    df['quality_status']
    .value_counts(normalize=True)
    .mul(100.0)
    .round(2)
    .reset_index()
    .rename(columns={'index': 'Quality Status', 'quality_status': 'Percentage'})
)

print("\n📊 Quality Status Distribution (% of unique labeled spans):")
print(quality_status_dist)


# Calculate percentage for 'answer' vs all other labels
# We use a case-insensitive comparison for 'answer'
answer_mask = df['label'].str.lower() == 'answer'
perc_answer = answer_mask.mean() * 100  # because the mean of booleans gives the proportion True
perc_non_answer = 100 - perc_answer
print("\n📊 'Answer' vs Other Labels (% of unique labeled spans):")
print(f"Answer: {perc_answer:.2f}%")
print(f"Other Labels: {perc_non_answer:.2f}%")


🔢 Total unique labeled spans: 594

📊 Label Distribution (% of unique labeled spans):
                  Percentage  proportion
0                     answer       44.95
1                    evasion       28.45
2                     attack        9.09
3             self_promotion        5.56
4           evasion + attack        4.21
5   evasion + self_promotion        3.87
6          evasion + stretch        1.85
7    attack + self_promotion        0.67
8           attack + stretch        0.67
9   self_promotion + stretch        0.51
10                   stretch        0.17

📊 Quality Status Distribution (% of unique labeled spans):
                                 Percentage  proportion
0                                 pure gold       62.12
1  result of low dice but containment check       26.09
2                         from disagreement       11.78

📊 'Answer' vs Other Labels (% of unique labeled spans):
Answer: 44.95%
Other Labels: 55.05%


Notes: 

- if prev_turn is empty, it's a marking in a question being asked

In [ ]:
#1. Subset df where PREV_MATCH_FLAG is "NO MATCH"
df_no_prev_match = df[df['PREV_MATCH_FLAG'] == "NO MATCH"]

# 2. Extract unique debate_unit_id values from this subset
debate_ids_no_match = df_no_prev_match['debate_unit_id'].unique()

# 3. Subset debates to include only rows with these debate_unit_ids
debates_subset = debates[debates['debate_unit_id'].isin(debate_ids_no_match)]


In [16]:
df.columns

Index(['debate_unit_id', 'label', 'span', 'quality_status',
       'source_annotators', 'construction_note', 'full_text', 'span_turn',
       'span_speaker', 'prev_turn', 'prev_turn_speaker', 'next_turn',
       'next_turn_speaker', 'METADATA_SessionID', 'METADATA_MeetingNumber',
       'METADATA_Date', 'METADATA_Location', 'METADATA_AgendaItemNo',
       'METADATA_AgendaTitle', 'METADATA_DebateType', 'METADATA_TurnNo',
       'METADATA_Speaker', 'METADATA_Party', 'METADATA_Role',
       'METADATA_TurnRole', 'METADATA_Time', 'METADATA_Utterance',
       'METADATA_AgendaCategory', 'METADATA_MeetingDateID',
       'METADATA_AgendaTitleDateID', 'METADATA_TurnRole_Danish', 'MATCH_FLAG',
       'METADATA_PREV_SessionID', 'METADATA_PREV_MeetingNumber',
       'METADATA_PREV_Date', 'METADATA_PREV_Location',
       'METADATA_PREV_AgendaItemNo', 'METADATA_PREV_AgendaTitle',
       'METADATA_PREV_DebateType', 'METADATA_PREV_TurnNo',
       'METADATA_PREV_Speaker', 'METADATA_PREV_Party', 'METADAT